In [ ]:
import os
import gc
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
DATA_DIRECTORY = ""

In [ ]:
train = pd.read_csv(os.path.join(DATA_DIRECTORY, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIRECTORY, 'test.csv'))
labels = pd.read_csv(os.path.join(DATA_DIRECTORY, 'labels.csv'))

In [ ]:
train_values = labels.to_numpy()
train_id = train['SK_ID_CURR']
test_id = test['SK_ID_CURR']

train_df_xg = train.copy()
test_df_xg = test.copy()

train_df_xg.drop('SK_ID_CURR', inplace=True, axis=1)
test_df_xg.drop('SK_ID_CURR', inplace=True, axis=1)

train_df_xg, test_df_xg = train_df_xg.align(test_df_xg, join = 'inner', axis = 1)

ratio = (train_values == 0).sum()/ (train_values == 1).sum()
del train, test
gc.collect()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df_xg, train_values, test_size=0.2, stratify=train_values, random_state=1)

In [ ]:
clf = XGBClassifier(
    n_estimators=1200, 
    objective='binary:logistic', 
    gamma=0.098, 
    subsample=0.5, 
    scale_pos_weight=ratio,
    )
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='auc', early_stopping_rounds=10)

In [ ]:
predictions = clf.predict_proba(test_df_xg.values)[:, 1]
submission = pd.DataFrame({'SK_ID_CURR': test_id.values, 'TARGET': predictions})
submission.to_csv('xgboost.csv', index = False)